# Modeling Tower Physics

## Rigid-body *statics without* block friction

Static equilibrium (absence external disturbances) is the most fundamental property that the tower must have in order for it to not topple. To start, masses and inertias of the blocks will be used to compute the net wrench
(force and moment) due to gravity, without considering frictional effects or minor deviations in the blocks’ nominal dimensions.

In [4]:
import numpy as np

In [7]:
# Step: Define indices of each block. 18 rows, 3 columns.
# i, j, k
max_k = 18
max_j = 3
max_i = 3
blockid = np.arange(1, max_k*max_i+1, 1)

# the indices are a 3-element list
# i and j are both row indices, but they determine the orientation of the block

block_data_struct = []
blockid_idx = 0
for idx1 in range(max_i):
    for idx2 in range(max_j):
        for idx3 in range(max_k):
            # if idx3 is odd, then idx1_val = 1, else idx1_val = idx1
            # if idx3 is even, then idx2_val = 1, else idx2_val = idx2
            if idx3 % 2 == 0:
                idx1_val = idx1+1
                idx2_val = 1
            else:
                idx1_val = 1
                idx2_val = idx2+1
            blockid_idx+=1
            block_data_struct.append([blockid_idx, idx1_val, idx2_val, idx3+1])

# function to compute center of mass of a single black
def single_block_com(i, j, k):
    # i, j, k are indices of the block, starting at 1
    # nominal parameters
    w = 0.04 # meters
    l = 0.07 # meters
    h = 0.02 # meters
    x_com = (i-1)*l/2
    return 0


[[1, 1, 1, 1], [2, 1, 1, 2], [3, 1, 1, 3], [4, 1, 1, 4], [5, 1, 1, 5], [6, 1, 1, 6], [7, 1, 1, 7], [8, 1, 1, 8], [9, 1, 1, 9], [10, 1, 1, 10], [11, 1, 1, 11], [12, 1, 1, 12], [13, 1, 1, 13], [14, 1, 1, 14], [15, 1, 1, 15], [16, 1, 1, 16], [17, 1, 1, 17], [18, 1, 1, 18], [19, 1, 1, 1], [20, 1, 2, 2], [21, 1, 1, 3], [22, 1, 2, 4], [23, 1, 1, 5], [24, 1, 2, 6], [25, 1, 1, 7], [26, 1, 2, 8], [27, 1, 1, 9], [28, 1, 2, 10], [29, 1, 1, 11], [30, 1, 2, 12], [31, 1, 1, 13], [32, 1, 2, 14], [33, 1, 1, 15], [34, 1, 2, 16], [35, 1, 1, 17], [36, 1, 2, 18], [37, 1, 1, 1], [38, 1, 3, 2], [39, 1, 1, 3], [40, 1, 3, 4], [41, 1, 1, 5], [42, 1, 3, 6], [43, 1, 1, 7], [44, 1, 3, 8], [45, 1, 1, 9], [46, 1, 3, 10], [47, 1, 1, 11], [48, 1, 3, 12], [49, 1, 1, 13], [50, 1, 3, 14], [51, 1, 1, 15], [52, 1, 3, 16], [53, 1, 1, 17], [54, 1, 3, 18], [55, 2, 1, 1], [56, 1, 1, 2], [57, 2, 1, 3], [58, 1, 1, 4], [59, 2, 1, 5], [60, 1, 1, 6], [61, 2, 1, 7], [62, 1, 1, 8], [63, 2, 1, 9], [64, 1, 1, 10], [65, 2, 1, 11], [66,